# Example on how Bloom / Hash Embedding works

This embedding uses single features of its inputs to create an embedding.
Each feature can be represented by an vector with M elements. The sum of all
features generates the input vector.

In [109]:
import numpy as np
import mmh3
import random

As you can see, with need embeddings for 20 words:

In [110]:
nouns = ['bunch', 'audience', 'flock', 'team', 'group', 'family', 
         'band', 'village', 'cat', 'sock', 'ship', 'hero',
         'monkey', 'baby', 'match', 'bed', 'dog', 'bottle', 'house',
         'paper']

len(nouns)

20

Let's say we choose 10 feature vectors, each with 2 elements, that can represent
a single word:

In [111]:
num_features = 10
feature_size = 2
feature_vecs = np.random.uniform(-0.1, 0.1, (num_features, feature_size))

print(feature_vecs)

[[-0.08058126 -0.03549474]
 [-0.02258513 -0.02622956]
 [-0.04844884  0.08611142]
 [-0.02237704  0.08822986]
 [-0.01117155 -0.00470919]
 [ 0.01774416 -0.03864823]
 [-0.0529064  -0.08787051]
 [-0.00538983 -0.00827288]
 [ 0.01222911  0.05121812]
 [ 0.01081429  0.06178878]]


Now we need to define a rule, that maps **one** input onto **M** of these vectors.
Normally we would check for specific word features (POS, position in text, ...).
In this example we use Hashes to map words onto a range from `0` to `num_features - 1`:

In [112]:
feature_keys_a = [mmh3.hash(n, 1) % num_features for n in nouns]
feature_keys_b = [mmh3.hash(n, 2) % num_features for n in nouns]
feature_keys_c = [mmh3.hash(n, 3) % num_features for n in nouns]

print(f'Inividual features for a: {len(set(feature_keys_a))}')
print(f'Inividual features for b: {len(set(feature_keys_b))}')
print(f'Inividual features for c: {len(set(feature_keys_c))}')

Inividual features for a: 7
Inividual features for b: 9
Inividual features for c: 10


As you can see, one feature alone can't differentiate between every single word.
There are collisions! Because of this we combine our features. This way, each word is defined
by three features:

In [113]:
feature_keys = list(zip(
    feature_keys_a,
    feature_keys_b,
    feature_keys_c,
))

print(f'Feature keys for first two inputs: {feature_keys[:2]}')
print(f'Individual combined features: {len(set(feature_keys))}')

Feature keys for first two inputs: [(3, 7, 1), (6, 0, 3)]
Individual combined features: 20


Now we can see, every word is represented by an individual tuple of keys.
In the last step we can search for the corresponding features and add them up:

In [114]:
embeddings = []
for keys in feature_keys:
    emb = sum(feature_vecs[k] for k in keys)
    embeddings.append(emb)

In [115]:
for word, emb in zip(nouns, embeddings):
    print(f'{word} -> {emb}')

bunch -> [-0.050352    0.05372742]
audience -> [-0.1558647  -0.03513538]
flock -> [-0.08361109 -0.04040732]
team -> [-0.09136091 -0.0520405 ]
group -> [-0.01002271  0.04130874]
family -> [0.00144945 0.03467235]
band -> [-0.05774738 -0.1527483 ]
village -> [-0.04606712 -0.04492528]
cat -> [-0.02152758  0.02027937]
sock -> [-0.14020165  0.0459075 ]
ship -> [-0.14980408  0.08435234]
hero -> [ 0.04630261 -0.01550768]
monkey -> [-0.09786857 -0.0258702 ]
baby -> [-0.02721801  0.02335207]
match -> [-0.04055207 -0.13479163]
bed -> [-0.16655234 -0.07926236]
dog -> [-0.11698436 -0.18045021]
bottle -> [-0.0592285   0.06956565]
house -> [-0.09320292  0.26257114]
paper -> [-0.05326366 -0.03079092]


Now each word has it's own representation, even though we only store 10 feature vectors!
However, this example has one flaw: Adding an unknown word may be possible, but it is dependent 
on a random hash function. So even if we train a model on these words, results for new
won't make any sense.

For example: We use this method to check if a word is a noun or verb. We may be able to train a model,
but for new words arbitrary hash values will be generated to get feature vectors. So it is not possible
to say if an unknown word is a verb or noun.

But what we can test is training a model on the known words

In [141]:
random.seed(0)

nb_epoch = 100
learn_rate = 0.1
    
labels = np.random.uniform(-0.1, 0.1, (len(nouns), 2))
examples = list(zip(nouns, feature_keys, labels))

for epoch in range(nb_epoch):
    random.shuffle(examples)
    loss=0.0

    for noun, keys, label in examples:
        hash_vector = sum(feature_vecs[k] for k in keys)

        diff = hash_vector - label

        for k in keys:
            feature_vecs[k] -= learn_rate * diff

        loss += (diff**2).sum()
    print(epoch, loss)

0 0.2584920620682681
1 0.20421630834396456
2 0.17985221189984488
3 0.16256476557363916
4 0.15651477342069486
5 0.1539980470577015
6 0.14941944216323205
7 0.15495934058180125
8 0.13974220357689598
9 0.14000019907864833
10 0.14364069543255248
11 0.14127378651259745
12 0.14851220255985334
13 0.14295166947298749
14 0.1423234540300131
15 0.13905704788288586
16 0.14183426626600465
17 0.14214815417937154
18 0.14772398381702004
19 0.14881169898958704
20 0.14152221190918393
21 0.14177297145108703
22 0.14733553376470787
23 0.1467140777521138
24 0.14022723117311342
25 0.13893245794794154
26 0.14065391558903745
27 0.1448032074734772
28 0.13824316863757646
29 0.15130812397214255
30 0.15009535187104273
31 0.14734792907675529
32 0.1470798201533985
33 0.14647412810438826
34 0.1417883194458527
35 0.1472935013360437
36 0.14658727754053555
37 0.14092185880827804
38 0.14400377664746145
39 0.14203383679365206
40 0.14495631797750988
41 0.14465548109431137
42 0.14027696147487811
43 0.1511848547541024
44 0.14